In [9]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import QuantLib as ql
import os

In [12]:
option_data = pd.read_csv(r'C:\Users\HP\Downloads\option_data_vn.csv')
option_data.drop(columns={'Unnamed: 0'}, inplace=True)
option_data['expire_date'] = pd.to_datetime(option_data['expire_date'], format = '%d/%m/%Y')
option_data['break_even_price'] = pd.to_numeric(option_data['break_even_price'], errors='coerce')
option_data['premium'] = pd.to_numeric(option_data['premium'], errors='coerce')
option_data.head()

,Ticker,break_even_price,convert_rate,expire_date,current_price,premium,diff,cost_per_call
0,CVNM2315,92.09,3.9199,2025-01-02,66.80,1.31,-25.29,5.135069
1,CVHM2307,62.91,8.0000,2024-06-24,39.95,0.03,-22.96,0.240000
2,CMSN2317,97.19,8.0000,2025-01-02,75.50,1.23,-21.69,9.840000
3,CVHM2311,61.10,5.0000,2024-08-01,39.95,0.13,-21.15,0.650000
4,CVNM2314,87.81,7.8398,2024-10-03,66.80,0.31,-21.01,2.430338


In [13]:
option_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Ticker            114 non-null    object        
 1   break_even_price  114 non-null    float64       
 2   convert_rate      114 non-null    float64       
 3   expire_date       114 non-null    datetime64[ns]
 4   current_price     114 non-null    float64       
 5   premium           114 non-null    float64       
 6   diff              114 non-null    float64       
 7   cost_per_call     114 non-null    float64       
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 7.2+ KB


In [ ]:
def black_scholes_call(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    delta = norm.cdf(d1)
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    theta = -(S * norm.pdf(d1) * sigma) / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2)
    vega = S * norm.pdf(d1) * np.sqrt(T)
    rho = K * T * np.exp(-r * T) * norm.cdf(d2)
    return call_price, delta, gamma, theta, vega, rho

In [4]:
v0 = 0.01
theta = 0.02
kappa = 0.2
sigma = 0.3
rho = 0.3
risk_free_rate = 0.0244
dividend_rate = 0.15

In [5]:
current_date = ql.Date(29, 5, 2024)
ql.Settings.instance().evaluationDate = current_date

In [6]:
day_count = ql.Actual365Fixed()

risk_free_curve = ql.FlatForward(
    current_date,
    ql.QuoteHandle(
        ql.SimpleQuote(risk_free_rate)
    ),
    day_count
)

dividend_curve = ql.FlatForward(
    current_date,
    ql.QuoteHandle(
        ql.SimpleQuote(dividend_rate)
    ),
    day_count
)

In [7]:
def calculate_implied_volatilities(df):
    implied_vols = []
    for index, row in df.iterrows():
        S = row['current_price']
        K = row['K']
        T = row['T']
        market_price = row['premium']
        if T == 0:
            implied_vols.append(0.0)
            continue
        expiration_date = ql.Date(29, 5, 2024) + int(T * 365)
        payoff = ql.PlainVanillaPayoff(ql.Option.Call, K)
        exercise = ql.EuropeanExercise(expiration_date)
        option = ql.VanillaOption(payoff, exercise)
        
        spot_handle = ql.QuoteHandle(ql.SimpleQuote(S))
        heston_process = ql.HestonProcess(
            ql.YieldTermStructureHandle(risk_free_curve),
            ql.YieldTermStructureHandle(dividend_curve),
            spot_handle,
            v0,
            kappa,
            theta,
            sigma,
            rho
        )
        heston_model = ql.HestonModel(heston_process)
        heston_engine = ql.AnalyticHestonEngine(heston_model)
        option.setPricingEngine(heston_engine)
        
        black_scholes_process = ql.GeneralizedBlackScholesProcess(
            spot_handle,
            ql.YieldTermStructureHandle(dividend_curve),
            ql.YieldTermStructureHandle(risk_free_curve),
            ql.BlackVolTermStructureHandle(ql.BlackConstantVol(current_date, ql.NullCalendar(), 0.2, day_count))
        )
        
        try:
            implied_vol = option.impliedVolatility(market_price, black_scholes_process)
        except RuntimeError:
            implied_vol = None
        
        implied_vols.append(implied_vol)
    return implied_vols

In [8]:
option_data['K'] = option_data['break_even_price'] - option_data['premium']
option_data['T'] = (option_data['expire_date'] - pd.to_datetime('2024-05-29')).dt.days/365.0
option_data['implied_volatility_heston'] = calculate_implied_volatilities(option_data)

KeyError: 'risk_free_rate'

In [ ]:
option_data.head()

,Ticker,break_even_price,convert_rate,expire_date,current_price,premium,diff,cost_per_call,K,T,implied_volatility_heston
0,CVNM2315,92.09,3.9199,2025-01-02,66.80,1.31,-25.29,5.135069,90.78,0.597260,0.411799
1,CVHM2307,62.91,8.0000,2024-06-24,39.95,0.03,-22.96,0.240000,62.88,0.071233,0.734875
2,CMSN2317,97.19,8.0000,2025-01-02,75.50,1.23,-21.69,9.840000,95.96,0.597260,0.343395
3,CVHM2311,61.10,5.0000,2024-08-01,39.95,0.13,-21.15,0.650000,60.97,0.175342,0.562682
4,CVNM2314,87.81,7.8398,2024-10-03,66.80,0.31,-21.01,2.430338,87.50,0.347945,0.326670


In [ ]:
# calculate value at expire
option_data['value_at_expiry'] = np.maximum(option_data['current_price'] - option_data['K'], 0)

In [ ]:
def buy_low_iv(df, threshold, max_options=5):
    low_iv_options = df[df['implied_volatility_heston'] < threshold]
    return low_iv_options.head(max_options)

def buy_in_the_money(df, max_options=5):
    in_the_money_options = df[df['current_price'] > df['K']]
    return in_the_money_options.head(max_options)

def buy_out_of_the_money(df, max_options=5):
    otm_options = df[df['current_price'] < df['K']]
    return otm_options.head(max_options)

def buy_near_the_money(df, max_options=5, tolerance=0.05):
    ntm_options = df[(df['current_price'] >= df['K'] * (1 - tolerance)) & (df['current_price'] <= df['K'] * (1 + tolerance))]
    return ntm_options.head(max_options)

def buy_high_delta(df, max_options=5):
    df['delta'] = (np.log(df['current_price'] / df['K']) + (risk_free_rate + 0.5 * df['implied_volatility_heston']**2) * df['T']) / (df['implied_volatility_heston'] * np.sqrt(df['T']))
    df['delta'] = norm.cdf(df['delta'])
    high_delta_options = df.sort_values(by='delta', ascending=False)
    return high_delta_options.head(max_options)

def buy_high_iv(df, threshold, max_options=5):
    high_iv_options = df[df['implied_volatility_heston'] > threshold]
    return high_iv_options.head(max_options)

In [ ]:
# Backtest strategies
def backtest_strategy(strategy, df, *args, max_options=5):
    selected_options = strategy(df, *args, max_options=max_options)
    portfolio_value = selected_options['value_at_expiry'].sum()
    total_cost = selected_options['premium'].sum()
    roi = (portfolio_value - total_cost) / total_cost if total_cost != 0 else np.nan
    return portfolio_value, total_cost, roi

In [ ]:
iv_threshold_low = 0.2
iv_threshold_high = 0.6

In [ ]:
results = {
    'Low Implied Volatility': backtest_strategy(buy_low_iv, option_data, iv_threshold_low, max_options=5),
    'In-The-Money': backtest_strategy(buy_in_the_money, option_data, max_options=5),
    'Out-Of-The-Money': backtest_strategy(buy_out_of_the_money, option_data, max_options=5),
    'Near-The-Money': backtest_strategy(buy_near_the_money, option_data, max_options=5),
    'High Delta': backtest_strategy(buy_high_delta, option_data, max_options=5),
    'High Implied Volatility': backtest_strategy(buy_high_iv, option_data, iv_threshold_high, max_options=5),
}

In [ ]:
for strategy, (portfolio_value, total_cost, roi) in results.items():
    print(f"{strategy} Portfolio:")
    print(f" - Portfolio Value at Expiry: {portfolio_value}")
    print(f" - Total Cost: {total_cost}")
    print(f" - ROI: {roi:.2%}")
    print('-'*30)

Low Implied Volatility Portfolio:
 - Portfolio Value at Expiry: 7.229999999999986
 - Total Cost: 10.91
 - ROI: -33.73%
------------------------------
In-The-Money Portfolio:
 - Portfolio Value at Expiry: 3.3299999999999876
 - Total Cost: 11.34
 - ROI: -70.63%
------------------------------
Out-Of-The-Money Portfolio:
 - Portfolio Value at Expiry: 0.0
 - Total Cost: 3.0100000000000002
 - ROI: -100.00%
------------------------------
Near-The-Money Portfolio:
 - Portfolio Value at Expiry: 0.0
 - Total Cost: 14.08
 - ROI: -100.00%
------------------------------
High Delta Portfolio:
 - Portfolio Value at Expiry: 12.820000000000004
 - Total Cost: 12.9
 - ROI: -0.62%
------------------------------
High Implied Volatility Portfolio:
 - Portfolio Value at Expiry: 0.0
 - Total Cost: 4.11
 - ROI: -100.00%
------------------------------


In [ ]:
final_portfolio_low_iv = buy_low_iv(option_data, iv_threshold_low, max_options=5)
final_portfolio_in_the_money = buy_in_the_money(option_data, max_options=5)
final_portfolio_otm = buy_out_of_the_money(option_data, max_options=5)
final_portfolio_ntm = buy_near_the_money(option_data, max_options=5)
final_portfolio_high_delta = buy_high_delta(option_data, max_options=5)
final_portfolio_high_iv = buy_high_iv(option_data, iv_threshold_high, max_options=5)

In [ ]:
final_portfolio_low_iv

,Ticker,break_even_price,convert_rate,expire_date,current_price,premium,diff,cost_per_call,K,T,implied_volatility_heston,value_at_expiry,delta
58,CFPT2317,139.85,15.0000,2024-11-19,137.00,2.60,-2.85,39.00000,137.25,0.476712,0.159382,0.00,0.557311
95,CACB2307,30.16,6.0000,2024-08-19,29.65,0.87,-0.51,5.22000,29.29,0.224658,0.198718,0.36,0.592892
99,CHPG2401,29.46,2.7268,2024-06-17,29.15,0.45,-0.31,1.22706,29.01,0.052055,0.180108,0.14,0.566949
100,CMWG2313,61.76,6.0000,2024-08-07,61.50,2.26,-0.26,13.56000,59.50,0.191781,0.190725,2.00,0.689211
107,CFPT2318,137.00,8.0000,2024-07-11,137.00,4.73,0.00,37.84000,132.27,0.117808,0.175066,4.73,0.746212


In [ ]:
final_portfolio_in_the_money

,Ticker,break_even_price,convert_rate,expire_date,current_price,premium,diff,cost_per_call,K,T,implied_volatility_heston,value_at_expiry,delta
62,CFPT2313,139.50,10.0000,2024-08-07,137.00,3.89,-2.50,38.900000,135.61,0.191781,0.203447,1.39,0.583763
72,CHPG2322,30.72,1.8179,2024-07-15,29.15,1.74,-1.57,3.163146,28.98,0.128767,0.458578,0.17,0.554451
75,CMBB2318,23.95,3.9138,2024-09-19,22.60,1.60,-1.35,6.262080,22.35,0.309589,0.389579,0.25,0.577126
76,CPDR2305,26.93,3.6127,2024-06-24,25.60,2.06,-1.33,7.442162,24.87,0.071233,0.674435,0.73,0.602714
77,CHPG2329,30.41,1.8179,2024-08-01,29.15,2.05,-1.26,3.726695,28.36,0.175342,0.417387,0.79,0.606062


In [ ]:
final_portfolio_otm

,Ticker,break_even_price,convert_rate,expire_date,current_price,premium,diff,cost_per_call,K,T,implied_volatility_heston,value_at_expiry,delta
0,CVNM2315,92.09,3.9199,2025-01-02,66.80,1.31,-25.29,5.135069,90.78,0.597260,0.411799,0.0,0.223954
1,CVHM2307,62.91,8.0000,2024-06-24,39.95,0.03,-22.96,0.240000,62.88,0.071233,0.734875,0.0,0.013700
2,CMSN2317,97.19,8.0000,2025-01-02,75.50,1.23,-21.69,9.840000,95.96,0.597260,0.343395,0.0,0.237001
3,CVHM2311,61.10,5.0000,2024-08-01,39.95,0.13,-21.15,0.650000,60.97,0.175342,0.562682,0.0,0.048631
4,CVNM2314,87.81,7.8398,2024-10-03,66.80,0.31,-21.01,2.430338,87.50,0.347945,0.326670,0.0,0.103752


In [ ]:
final_portfolio_ntm

,Ticker,break_even_price,convert_rate,expire_date,current_price,premium,diff,cost_per_call,K,T,implied_volatility_heston,value_at_expiry,delta
32,CFPT2314,144.60,10.0000,2025-01-07,137.00,4.46,-7.60,44.600000,140.14,0.610959,0.234875,0.0,0.519764
39,CHPG2339,34.05,1.8179,2025-01-02,29.15,3.75,-4.90,6.817125,30.30,0.597260,0.602664,0.0,0.571852
50,CMWG2310,65.00,5.0000,2024-08-01,61.50,2.00,-3.50,10.000000,63.00,0.175342,0.314427,0.0,0.466251
58,CFPT2317,139.85,15.0000,2024-11-19,137.00,2.60,-2.85,39.000000,137.25,0.476712,0.159382,0.0,0.557311
60,CHPG2338,31.70,3.6358,2024-10-03,29.15,1.27,-2.55,4.617466,30.43,0.347945,0.340157,0.0,0.471484


In [ ]:
final_portfolio_high_delta

,Ticker,break_even_price,convert_rate,expire_date,current_price,premium,diff,cost_per_call,K,T,implied_volatility_heston,value_at_expiry,delta
108,CMBB2401,22.58,1.9569,2024-06-17,22.60,1.56,0.02,3.052764,21.02,0.052055,0.297212,1.58,0.868948
109,CTCB2312,46.72,2.9095,2024-07-11,46.90,5.19,0.18,15.100305,41.53,0.117808,0.346309,5.37,0.865776
106,CVIB2401,22.32,1.8879,2024-06-17,22.30,1.30,-0.02,2.454270,21.02,0.052055,0.285613,1.28,0.831272
103,CVIB2306,22.47,1.8879,2024-08-21,22.30,3.50,-0.17,6.607650,18.97,0.230137,0.483223,3.33,0.798926
104,CHPG2343,29.24,2.7268,2024-06-12,29.15,1.35,-0.09,3.681180,27.89,0.038356,0.288778,1.26,0.795630


In [ ]:
final_portfolio_high_iv

,Ticker,break_even_price,convert_rate,expire_date,current_price,premium,diff,cost_per_call,K,T,implied_volatility_heston,value_at_expiry,delta
1,CVHM2307,62.91,8.0000,2024-06-24,39.95,0.03,-22.96,0.240000,62.88,0.071233,0.734875,0.0,0.013700
6,CVIC2306,63.42,8.0000,2024-06-24,45.05,0.10,-18.37,0.800000,63.32,0.071233,0.680639,0.0,0.038065
19,CVRE2308,32.73,5.0000,2024-06-24,22.45,0.13,-10.28,0.650000,32.60,0.071233,0.870368,0.0,0.069148
25,CVRE2318,31.28,4.0000,2024-07-03,22.45,0.10,-8.83,0.400000,31.18,0.095890,0.652412,0.0,0.065095
39,CHPG2339,34.05,1.8179,2025-01-02,29.15,3.75,-4.90,6.817125,30.30,0.597260,0.602664,0.0,0.571852
